# Generador de archivos para llenar buckets en GCP

Imports

In [1]:
import pandas as pd
import random as rd
import string as st
import os
import google.cloud.storage as storage
from datetime import datetime
from datetime import timedelta

Constantes

In [18]:
# Rutas
path = '../data/TP_Final_Especificacion_1.xlsx'
path_coord ='../data/ar.csv'
path_destino = '../data'

# Cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 2

# Fecha actual
fecha_actual = datetime.now()

# Cantidad de distribuidores
cant_dist = 4 # 3 - 1

# Cantidad de clientes
cant_clientes = 4

# Cantidad de sucursales
cant_sucursales = 5

Usamos credenciales e instanciamos storage cliente

In [4]:
credentials = '../usm-infra-grupo1-42c8c1c4178a.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials
storage_client = storage.Client()

Leemos hojas de planilla excel, generamos dataframes de cada una y leemos csv de coordenadas

In [7]:
df_sheets_desc = pd.read_excel(path, sheet_name=None)
df_cond_vta = df_sheets_desc['Condicion_Venta']
df_unidades = df_sheets_desc['Unidades']
df_tipo_negocio = df_sheets_desc['Tipo_Negocio']
df_cadena = df_sheets_desc['Cadena']
df_dias_visita = df_sheets_desc['Dias_Visita']
df_productos = df_sheets_desc['Productos']
df_cond_venta = df_sheets_desc['Condicion_Venta']
df_localidades = df_sheets_desc['Prov_Localidades']
df_estado = df_sheets_desc['Estado_cliente']
df_dias_visita = df_sheets_desc['Dias_Visita']

coordenadas = pd.read_csv(path_coord)

Funciones útiles

In [8]:
def generar_codigos_sku(productos):
  codigos_sku = {}
  for producto in productos:
    caracteres = st.ascii_uppercase + st.digits
    sku_codigo = ''.join(rd.choice(caracteres) for _ in range(10))
    codigos_sku[producto] = sku_codigo

  return codigos_sku

In [12]:
def generate_fecha_cierre(fecha_cierre: datetime) -> str:
    """
    Genera una fecha de cierre de comprobante aleatoria entre la fecha de cierre
    de comprobante y la fecha actual.
    """
    fech_cie_com = (fecha_cierre - timedelta(hours=rd.randint(0, 23), minutes=rd.randint(0, 59), seconds=rd.randint(0, 59))) \
            .strftime('%Y-%m-%d %H:%M:%S')
    return fech_cie_com

In [15]:
def generate_data_per_days(
        days: int,
        distribuidor: int,
        df_cond_vta: pd.DataFrame,
        df_localidades: pd.DataFrame,
        df_dias_visita: pd.DataFrame,
        df_productos: pd.DataFrame,
        df_sheets_desc: dict,
        df_tipo_negocio: pd.DataFrame,
        df_estado: pd.DataFrame,
        coordenadas: pd.DataFrame,
    ) -> tuple[dict, str]:
    fech_cierre = fecha_actual - timedelta(days=days)
    fech_cierre_string = fech_cierre.strftime('%Y-%m-%d')
    fech_cie_com_list = []
    vta_unidades_list = []
    vta_importe_list = []
    cond_vta_list = []
    sucursales_dist_list = []
    clientes_list = []
    provincias_list = []
    ciudades_list = []
    estado_list = []
    nombre_clientes_list = []
    cuit_list = []
    razon_social_list = []
    direccion_list = []
    dias_visita_list = []
    telefono_list = []
    fecha_alta_list = []
    fecha_baja_list = []
    tipo_negocio_list = []
    unidad_list = []
    stock_list = []
    deuda_vencida_list = []
    deuda_total_list = []

    # Generar SKUs a partir de la lista de productos
    codigos_sku = generar_codigos_sku(df_productos['nombre'])

    # Nombre de las columnas de los archivos cliente,venta,stock
    columns_v = df_sheets_desc['Desc_Ventas']['Campo']
    columns_s = df_sheets_desc['Desc_Stock']['Campo']
    columns_m = df_sheets_desc['Desc_Clientes']['Campo']
    columns_d = df_sheets_desc['Desc_Deuda']['Campo']

    for cliente_nro in range(cant_clientes):
        codigo_sucursal_base = (distribuidor * 100)  # Código base para el distribuidor
        fech_cie_com_list.append(generate_fecha_cierre(fecha_cierre=fech_cierre))
        vta_unidades_list.append(rd.randint(0,100))
        vta_importe_list.append(round((rd.uniform(100, 1000)), 2))
        cond_vta_list.append(rd.choice(df_cond_vta['codigo_condicion']))
        sucursales_dist_list.append(codigo_sucursal_base + rd.randint(10, 99))
        clientes_list.append(codigo_sucursal_base + rd.randint(1000, 9999))
        provincias_list.append(rd.choice(df_localidades.columns.values))
        ciudades_list = [rd.choice(df_localidades[p].dropna().tolist()) for p in provincias_list]
        estado = rd.choices(df_estado['nombre'], weights=df_estado['probabilidades'], k=1)[0]
        estado_list.append(estado)
        nombre_clientes_list.append(f'Cliente_{cliente_nro}')
        cuit_list.append(rd.randint(10000000, 99999999))
        razon_social_list.append(f'Empresa_{cliente_nro}')
        direccion_list.append(f'Direccion_{cliente_nro}')
        dias_visita_list.append(rd.choice(df_dias_visita['codigo_dia']))
        telefono_list.append(f"{rd.randint(100, 999)}-{rd.randint(100, 999)}-{rd.randint(1000, 9999)}")
        if estado == "I":
            fecha_baja_list.append((fech_cierre + timedelta(days=rd.randint(cant_dias,365))).strftime('%Y-%m-%d'))
        else:
            fecha_baja_list.append(pd.NA)
        fecha_alta_list.append((fech_cierre - timedelta(days=rd.randint(0,cant_dias))).strftime('%Y-%m-%d'))
        tipo_negocio_list.append(rd.choice(df_tipo_negocio['nombre']))
        unidad = rd.choice(df_unidades['codigo_unidad'])
        unidad_list.append(unidad)
        stock_list.append(rd.randint(100, 500) if unidad == "UNI" else rd.randint(1, 100))
        deuda_vencida_list.append(round(rd.uniform(0, 1000), 2))
        deuda_total_list.append(round(rd.uniform(0, 10000), 2))
    
    coordenada_latitud = coordenadas[coordenadas['city'].isin(ciudades_list)]['lat']
    coordenada_longitud = coordenadas[coordenadas['city'].isin(ciudades_list)]['lng']

    #Producto
    producto=[]
    producto1=[]
    producto1 = list(codigos_sku.keys())
    producto1.extend(producto1)
    producto=producto1

    #SKU
    sku_codigo=[]
    sku_codigo1=[]
    sku_codigo1 = list(codigos_sku.values())
    sku_codigo1.extend(sku_codigo1)
    sku_codigo = sku_codigo1

    #data frame con data
    venta_clientes_data = list(
        zip(
            sucursales_dist_list,
            clientes_list,
            fech_cie_com_list,
            sku_codigo,
            vta_unidades_list,
            vta_importe_list,
            cond_vta_list
        )
    )
    stock_data = list(
        zip(
            sucursales_dist_list,
            fech_cie_com_list,
            sku_codigo,
            producto,
            stock_list,
            unidad_list
        )
    )
    clientes_data = list(
        zip(
            sucursales_dist_list,
            clientes_list,
            ciudades_list,
            provincias_list,
            estado_list,
            nombre_clientes_list,
            cuit_list,
            razon_social_list,
            direccion_list,
            dias_visita_list,
            telefono_list,
            fecha_alta_list,
            fecha_baja_list,
            coordenada_latitud,
            coordenada_longitud,
            cond_vta_list,
            deuda_vencida_list,
            tipo_negocio_list
        )
    )
    deuda_data = list(
        zip(
            sucursales_dist_list,
            clientes_list,
            fech_cie_com_list,
            deuda_vencida_list,
            deuda_total_list
        )
    )

    df_vta_cli = pd.DataFrame(venta_clientes_data,columns=columns_v)
    df_stock = pd.DataFrame(stock_data,columns=columns_s)
    df_clientes = pd.DataFrame(clientes_data,columns=columns_m)
    df_deuda =  pd.DataFrame(deuda_data,columns=columns_d)

    return { 'venta': df_vta_cli, 'stock': df_stock, 'cliente': df_clientes, 'deuda': df_deuda }, fech_cierre_string
    

In [19]:
for distribuidor in range(1, cant_dist):
    carpeta_destino = 'g1_distribuidor_' + str(distribuidor)
    bucket = storage_client.bucket(carpeta_destino)

    if not bucket.exists():
        bucket = storage_client.create_bucket(carpeta_destino)

    for dias in range(0, cant_dias):
        dfDict, fecha_cierre = generate_data_per_days(
            days=dias,
            distribuidor=distribuidor,
            df_cond_vta=df_cond_vta,
            df_localidades=df_localidades,
            df_dias_visita=df_dias_visita,
            df_productos=df_productos,
            df_sheets_desc=df_sheets_desc,
            df_tipo_negocio=df_tipo_negocio,
            df_estado=df_estado,
            coordenadas=coordenadas,
        )
        
        for key, df in dfDict.items():
            archivo_destino = f'{key}/{fecha_cierre}.csv'
            blob = bucket.blob(archivo_destino)
            blob.upload_from_string(df.to_csv(), 'text/csv')